![Notebook Banner](llama3_banner.png)

## Set Up Docker Login (Executing on A Terminal Locally)

Replace <your api key> with your own NGC API Key

```bash
export NGC_CLI_API_KEY="nvapi-xxx"
echo "$NGC_CLI_API_KEY" | docker login nvcr.io --username '$oauthtoken' --password-stdin
```

## Deploy NIM (Executing on this Jupyter Notebook)

In [ ]:
%%bash

export NGC_API_KEY="nvapi-xxx"

export CONTAINER_NAME="Llama3-8B-Instruct"
export IMG_NAME="nvcr.io/nim/meta/llama3-8b-instruct:1.0.3"
export LOCAL_NIM_CACHE=~/.cache/nim
mkdir -p $LOCAL_NIM_CACHE

docker run -d --rm --name=$CONTAINER_NAME \
    --runtime=nvidia \
    --gpus all \
    --shm-size=16GB \
    -e NGC_API_KEY \
    -v $LOCAL_NIM_CACHE:/opt/nim/.cache \
    -u $(id -u) \
    -p 8000:8000 \
    $IMG_NAME

f64fed02a76bd744005beefea1a8a1f2707b95fb0eaff03e3294d6ec19f7adb7


## Start Inferencing


In [23]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
from openai import OpenAI
client = OpenAI(
    # base_url='https://integrate.api.nvidia.com/v1',
    base_url='http://0.0.0.0:8000/v1',
    api_key="$NGC_API_KEY"
)
completion = client.chat.completions.create(
    model='meta/llama3-8b-instruct',
    messages=[{"role":"user", "content":"Write a limerick about the wonders of GPU computing."}],
    temperature=0.5,
    top_p=1,
    max_tokens=1024,
    stream=True
)
for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

There once was a GPU so fine,
Whose computing powers were truly divine.
It processed with zest,
Complex tasks with the best,
And made calculations that were sublime.

## Stop the Container

In [30]:
!docker ps -a && \
docker stop Llama3-8B-Instruct

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
Error response from daemon: No such container: Llama3-8B-Instruct
